In [5]:
import uuid
import random
from faker import Faker

In [6]:
supabase_url = "https://your-project-url.supabase.co"
supabase_key = "your-anon-key"
client = supabase.create_client(supabase_url, supabase_key)

# Fetch current table names
tables = client.rpc("pg_tables").execute()
for table in tables.data:
    print(table["tablename"])

In [ ]:
import requests

def get_table_columns(table_name):
    """Fetch column names and types from Supabase"""
    query = f"""
    SELECT column_name, data_type FROM information_schema.columns 
    WHERE table_name = '{table_name}'
    """
    response = client.rpc("pg_query", {"query": query}).execute()
    return response.data if response else []

# Example usage:
print(get_table_columns("persons"))


In [ ]:
import uuid
import random
from faker import Faker

fake = Faker()

def insert_data(table_name, num_rows=10):
    """Insert data into a given table dynamically"""
    columns = get_table_columns(table_name)
    
    if not columns:
        print(f"Table {table_name} does not exist or has no columns.")
        return

    data = []
    for _ in range(num_rows):
        row = {}
        for column in columns:
            col_name = column["column_name"]
            col_type = column["data_type"]
            
            # Generate appropriate fake data
            if "uuid" in col_type:
                row[col_name] = str(uuid.uuid4())
            elif "text" in col_type:
                row[col_name] = fake.word()
            elif "integer" in col_type:
                row[col_name] = random.randint(1, 100)
            elif "timestamp" in col_type:
                row[col_name] = fake.date_time().isoformat()
        
        data.append(row)

    # Insert into Supabase
    client.table(table_name).insert(data).execute()
    print(f"Inserted {num_rows} rows into {table_name}.")

# Example usage
insert_data("persons", num_rows=5)
